# Book Reading Propensity: Filtering the Data

In [1]:
import pandas as pd
import json
import os

## Filter the Data: 

In [2]:
data_file = "./data/goodreads_interactions_fantasy_paranormal.json"
data_file_filtered = "./data/goodreads_interactions_fantasy_paranormal_2016.json"


In [3]:
cnt = 0
with open(data_file, "r") as f:
    line = f.readline()
    print(line)
    cnt += 1
    # if cnt > 10:
    #     break

{"user_id": "8842281e1d1347389f2ab93d60773d4d", "book_id": "19161852", "review_id": "4443cb6883624c3772625ef5b7b4e138", "is_read": false, "rating": 0, "review_text_incomplete": "", "date_added": "Fri Sep 08 10:44:24 -0700 2017", "date_updated": "Fri Sep 08 10:44:24 -0700 2017", "read_at": "", "started_at": ""}



In [15]:
%%time

cnt = 0
cnt_relevant = 0

with open(data_file, "r") as fr:
    with open(data_file_filtered, "w") as fw:
        for line in fr:
            # print(line)
            cnt += 1
            try:
                record = json.loads(line)
            except Exception as e:
                print(f'Failed to convert the line to a dict:\n{line}')
                print('Exception ----------------------------- \n', e)
                continue
            
            date_updated = record.get("date_updated", "")
            read_at = record.get("read_at", "")
            combo = date_updated + ' ' + read_at
            if '2016' in combo:
                try:
                    json.dump(record, fw)
                    fw.write('\n')
                except Exception as e:
                    print(f'Failed to write the line to file:\n{line}')
                    print('Exception ----------------------------- \n', e)
                    continue
                cnt_relevant += 1
                if cnt_relevant != 0 and cnt_relevant % 1000000 == 0:
                    print(f'Relevant Record Count: {cnt_relevant:>10,d}. Total records inspected: {cnt:>10,d}')
                


            # if cnt > 30:
            #     break
print('-'*70)
print(f'Final Record Count (Input):           {cnt:>10,d}')
print(f'Final Relevant Record Count (Output): {cnt_relevant:10,d}')
print('-'*70)

Relevant Record Count:  1,000,000. Total records inspected:  7,191,867
Relevant Record Count:  2,000,000. Total records inspected: 13,981,481
Relevant Record Count:  3,000,000. Total records inspected: 20,156,727
Relevant Record Count:  4,000,000. Total records inspected: 25,717,217
Relevant Record Count:  5,000,000. Total records inspected: 30,759,259
Relevant Record Count:  6,000,000. Total records inspected: 35,226,872
Relevant Record Count:  7,000,000. Total records inspected: 38,337,797
Relevant Record Count:  8,000,000. Total records inspected: 42,209,869
Relevant Record Count:  9,000,000. Total records inspected: 50,634,669
----------------------------------------------------------------------
Final Record Count (Input):           55,397,550
Final Relevant Record Count (Output):  9,779,053
----------------------------------------------------------------------
CPU times: user 5min 5s, sys: 6.76 s, total: 5min 12s
Wall time: 5min 17s


In [16]:
# TEST

count = 0
with open(data_file_filtered, "r") as f:
    for i, line in enumerate(f):
        count = i
        if i < 3:
            print(f'Record # {i}')
            print(line)
print(f'Final Record Count: {count}')

Record # 0
{"user_id": "8842281e1d1347389f2ab93d60773d4d", "book_id": "17554595", "review_id": "c6fcd1758f10474be583c6c0bea2bc07", "is_read": false, "rating": 0, "review_text_incomplete": "", "date_added": "Tue Dec 06 12:08:34 -0800 2016", "date_updated": "Tue Dec 06 12:08:35 -0800 2016", "read_at": "", "started_at": ""}

Record # 1
{"user_id": "8842281e1d1347389f2ab93d60773d4d", "book_id": "18423", "review_id": "626f7f3a77c4aa1bd0c8f868966f2f20", "is_read": false, "rating": 0, "review_text_incomplete": "", "date_added": "Fri Oct 28 14:44:57 -0700 2016", "date_updated": "Fri Oct 28 14:44:57 -0700 2016", "read_at": "", "started_at": ""}

Record # 2
{"user_id": "8842281e1d1347389f2ab93d60773d4d", "book_id": "29058155", "review_id": "3759ba2b18dd09eb5e844a116f0ce06a", "is_read": true, "rating": 3, "review_text_incomplete": "", "date_added": "Mon Jul 25 14:29:33 -0700 2016", "date_updated": "Tue Dec 13 11:00:02 -0800 2016", "read_at": "Sat Aug 20 00:00:00 -0700 2016", "started_at": "Wed Au